# DNF 101: Dynamic neural fields in Lava

## Basic populations and connections

Create populations of leaky integrate-and-fire (LIF) neurons.
The `shape` argument determines the number of neurons and their layout.

In [ ]:
from lava.lib.dnf.population.process import Population

population = Population(shape=20)

Create connections between populations using the `connect()` function.
The weights can be specified using a sequence of operations (see below).

In [ ]:
from lava.lib.dnf.population.process import Population
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights

population1 = Population(shape=20)
population2 = Population(shape=20)
connect(population1.s_out, population2.a_in, [Weights(20)])

## Dynamic neural fields (DNF)

### Multi-peak DNF

Create dynamic neural fields (DNFs) that support multiple peaks by using the
`MultiPeakKernel` with local excitation and mid-range inhibition. Use the
`Convolution` operation to apply the kernel.

In [ ]:
from lava.lib.dnf.population.process import Population
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.connect.connect import connect

dnf = Population(shape=20)

kernel = MultiPeakKernel(amp_exc=25,
                         width_exc=3,
                         amp_inh=-15,
                         width_inh=6)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

### Selective DNF

Create DNFs that are selective and only create a single peak by using the
`SelectiveKernel` with local excitation and global inhibition.

In [ ]:
from lava.lib.dnf.population.process import Population
from lava.lib.dnf.kernels.kernels import SelectiveKernel
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.connect.connect import connect

dnf = Population(shape=20)

kernel = SelectiveKernel(amp_exc=18,
                         width_exc=3,
                         global_inh=-15)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

## Input

### Spike generators

To simulate spike input to a DNF, use a `RateCodeSpikeGen` Process. It
generates spikes with a spike rate pattern that can be specified, for
instance by using the `GaussPattern` Process. Connect the `RateCodeSpikeGen` to
 a DNF with the `connect()` function. You may change parameters of the
 `GaussPattern` during runtime.

In [ ]:
from lava.lib.dnf.population.process import Population
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.spike_generator.process import RateCodeSpikeGen
from lava.lib.dnf.operations.operations import Weights
from lava.lib.dnf.connect.connect import connect

shape = 15
# produces a pattern of spike rates for the spike generator
gauss_pattern = GaussPattern(shape=shape, amplitude=100, mean=5, stddev=5)
# produces spikes based on the given spike rates
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.a_out.connect(spike_generator.a_in)

# connect spike generator to a population
dnf = Population(shape=20)
connect(spike_generator.s_out, dnf.a_in, [Weights(20)])

# [...start running the network...] (see below)

gauss_pattern.amplitude = 50  # you may change the parameters during runtime

## Higher dimensions

Define DNFs and inputs over higher dimensionalities by specifying a `shape` with
multiple entries.

In [ ]:
shape = (15, 15)
dnf = Population(shape=shape)

Inputs and kernels must match the dimensionality of the DNF; specify
parameters such as `width_exc` as vectors rather than scalars.

In [ ]:
gauss_pattern = GaussPattern(shape=shape,
                             amplitude=100,
                             mean=[5, 5],
                             stddev=[4, 4])
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.s_out.connect(spike_generator.a_in)

kernel = MultiPeakKernel(amp_exc=58,
                         width_exc=[3.8, 3.8],
                         amp_inh=-50,
                         width_inh=[7.5, 7.5])
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

connect(spike_generator.s_out, dnf.a_in, [Weights(20)])

## Larger architectures

### One-to-one connections
When connecting two DNFs that have the same shape (in terms of neurons and
dimensions), use the operation `Weights`. It connects each neuron in the
first DNF to its (single) respective neuron in the second DNF.

In [ ]:
dnf1 = Population(shape=(10,))
dnf2 = Population(shape=(10,))

connect(dnf1, dnf2, [Weights(40)])

### Reducing dimensions
When the dimensionality of the source DNF is larger than that of
the target DNF, use the `ReduceDims` operation, specifying the indices of the
 dimensions that should be removed and how to remove them.

In [ ]:
from lava.lib.dnf.operations.operations import ReduceDims

dnf_2d = Population(shape=(20, 10,))
dnf_1d = Population(shape=(20,))

connect(dnf_2d.s_out, dnf_1d.a_in, [Weights(40), ReduceDims(reduce_dims=0)])

### Expanding dimensions
When the dimensionality of the source DNF is smaller than that of the target
DNF, use the `ExpandDims` operation, speciying the number of neurons of the
dimensions that will be added.

In [ ]:
from lava.lib.dnf.operations.operations import ExpandDims

dnf_1d = Population(shape=(20,))
dnf_2d = Population(shape=(20, 10))

connect(dnf_2d.s_out, dnf_1d.a_in, [Weights(40),
                                    ExpandDims(new_dims_shape=10)])

### Reordering dimensions
To reorder dimensions, use the `Reorder` operation, specifying the new order
of the indices.

In [ ]:
from lava.lib.dnf.operations.operations import Reorder

dnf_1d = Population(shape=(10, 20))
dnf_2d = Population(shape=(20, 10))

connect(dnf_2d.s_out, dnf_1d.a_in, [Reorder(order=(1, 0))])

Reordering can also be combined with the `ReduceDims` or `ExpandDims`
operation, as shown below.

In [ ]:
dnf_1d = Population(shape=(10,))
dnf_2d = Population(shape=(20, 10))

connect(dnf_2d.s_out, dnf_1d.a_in, [ExpandDims(new_dims_shape=20),
                                    Reorder(order=(1, 0))])

## Running and plotting networks

Call the `run()` method, specifying the number of time steps to run for and the
backend the model should run on. To enable plots, create probes before running
and create plots with the probed data after running.

In [ ]:
from lava.proc.io.monitor.process import Monitor
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.dnf.population.process import Population
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.spike_generator.process import RateCodeSpikeGen
from lava.lib.dnf.operations.operations import Weights, Convolution
from lava.lib.dnf.connect.connect import connect

shape = 15

gauss_pattern = GaussPattern(shape=shape, amplitude=100, mean=5, stddev=5)
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.a_out.connect(spike_generator.a_in)

dnf = Population(shape=shape)

kernel = MultiPeakKernel(amp_exc=17,
                         width_exc=3,
                         amp_inh=-15,
                         width_inh=6)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

connect(spike_generator.s_out, dnf.a_in, [Weights(20)])

monitor = Monitor()
monitor.probe(dnf.s_out)
monitor.probe(spike_generator.s_out)

spike_generator.run(condition=RunSteps(num_steps=100),
                    run_cfg=Loihi1SimCfg(select_sub_proc_model=True))

# [...plot monitor...]

spike_generator.stop()

## DNF instabilities

The following examples demonstrate the detection instability, the selection
instability, and the working memory regime with one-dimensional DNFs.

### Detection

In [ ]:
from lava.proc.io.monitor.process import Monitor
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.dnf.population.process import Population
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.spike_generator.process import RateCodeSpikeGen
from lava.lib.dnf.operations.operations import Weights, Convolution
from lava.lib.dnf.connect.connect import connect

shape = 15

gauss_pattern = GaussPattern(shape=shape, amplitude=0, mean=3.75, stddev=2.25)
spike_generator = RateCodeSpikeGen(shape=shape)
gauss_pattern.a_out.connect(spike_generator.a_in)

dnf = Population(shape=shape)

kernel = MultiPeakKernel(amp_exc=82,
                         width_exc=3.75,
                         amp_inh=-70,
                         width_inh=7.5)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

connect(spike_generator.s_out, dnf.a_in, [Weights(20)])

monitor = Monitor()
monitor.probe(dnf.s_out)
monitor.probe(spike_generator.s_out)

condition = RunSteps(num_steps=100)
run_cfg = Loihi1SimCfg(select_sub_proc_model=True)
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern.amplitude = 2300
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern.amplitude = 11200
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern.amplitude = 2300
spike_generator.run(condition=RunSteps(num_steps=200), run_cfg=run_cfg)
gauss_pattern.amplitude = 0
spike_generator.run(condition=condition, run_cfg=run_cfg)
spike_generator.stop()

# [...plot monitor...]

### Selection

In [ ]:
from lava.proc.io.monitor.process import Monitor
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.dnf.population.process import Population
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.spike_generator.process import RateCodeSpikeGen
from lava.lib.dnf.operations.operations import Weights, Convolution
from lava.lib.dnf.connect.connect import connect

shape = 15

gauss_pattern_1 = GaussPattern(shape=shape,
                               amplitude=0,
                               mean=11.25,
                               stddev=2.25)
spike_generator_1 = RateCodeSpikeGen(shape=shape)
gauss_pattern_1.a_out.connect(spike_generator_1.a_in)

gauss_pattern_2 = GaussPattern(shape=shape,
                               amplitude=0,
                               mean=3.75,
                               stddev=2.25)
spike_generator_2 = RateCodeSpikeGen(shape=shape)
gauss_pattern_2.a_out.connect(spike_generator_2.a_in)

dnf = Population(shape=shape)

kernel = SelectiveKernel(amp_exc=18,
                         width_exc=2.25,
                         global_inh=-15)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

connect(spike_generator_1.s_out, dnf.a_in, [Weights(30)])
connect(spike_generator_2.s_out, dnf.a_in, [Weights(30)])

monitor = Monitor()
monitor.probe(dnf.s_out)
monitor.probe(spike_generator_1.s_out)
monitor.probe(spike_generator_2.s_out)

run_cfg = Loihi1SimCfg(select_sub_proc_model=True)

dnf.run(condition=RunSteps(num_steps=99), run_cfg=run_cfg)
gauss_pattern_1.amplitude = 10000
dnf.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
gauss_pattern_2.amplitude = 10000
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
gauss_pattern_1.amplitude = 0
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
gauss_pattern_1.amplitude = 10000
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
gauss_pattern_2.amplitude = 0
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
gauss_pattern_2.amplitude = 10000
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
gauss_pattern_1.amplitude = 0
gauss_pattern_2.amplitude = 0
dnf.run(condition=RunSteps(num_steps=100), run_cfg=run_cfg)
dnf.stop()

# [...plot monitor...]

### Memory

In [ ]:
from lava.proc.io.monitor.process import Monitor
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.dnf.population.process import Population
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.spike_generator.process import RateCodeSpikeGen
from lava.lib.dnf.operations.operations import Weights, Convolution
from lava.lib.dnf.connect.connect import connect

shape = 15

gauss_pattern_1 = GaussPattern(shape=shape,
                               amplitude=0,
                               mean=11.25,
                               stddev=2.25)
spike_generator_1 = RateCodeSpikeGen(shape=shape)
gauss_pattern_1.a_out.connect(spike_generator_1.a_in)

gauss_pattern_2 = GaussPattern(shape=shape,
                               amplitude=0,
                               mean=3.75,
                               stddev=2.25)
spike_generator_2 = RateCodeSpikeGen(shape=shape)
gauss_pattern_2.a_out.connect(spike_generator_2.a_in)

dnf = Population(shape=shape)

kernel = MultiPeakKernel(amp_exc=30,
                         width_exc=2.5,
                         amp_inh=-18,
                         width_inh=4.5)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

connect(spike_generator_1.s_out, dnf.a_in, [Weights(20)])
connect(spike_generator_2.s_out, dnf.a_in, [Weights(20)])

monitor = Monitor()
monitor.probe(dnf.s_out)
monitor.probe(spike_generator.s_out)

condition = RunSteps(num_steps=100)
run_cfg = Loihi1SimCfg(select_sub_proc_model=True)

spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern_1.amplitude = 2300
gauss_pattern_2.amplitude = 2300
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern_1.amplitude = 11200
gauss_pattern_2.amplitude = 11200
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern_1.amplitude = 2300
gauss_pattern_2.amplitude = 2300
spike_generator.run(condition=condition, run_cfg=run_cfg)
gauss_pattern_1.amplitude = 0
gauss_pattern_2.amplitude = 0
spike_generator.run(condition=condition, run_cfg=run_cfg)
spike_generator.stop()

# [...plot monitor...]